<a href="https://colab.research.google.com/github/lgregs/TCC-CD-USP/blob/main/create_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import zipfile
import os
import re

In [2]:
zip_file_path = '/content/drive/MyDrive/TCC/balance_sheets_dataset (1).zip'
extract_dir = 'extracted_balance_sheets'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Arquivos extraídos para: {extract_dir}")
print(f"Primeiros 5 arquivos extraídos: {os.listdir(extract_dir)[:5]}")

Arquivos extraídos para: extracted_balance_sheets
Primeiros 5 arquivos extraídos: ['EMNIS_2009.xlsx', 'VKFYO_2008.xlsx', 'KOTON_2024.xlsx', 'KRTEK_2023.xlsx', 'PNSUT_2009.xlsx']


In [3]:
all_data = []
# Certifique-se de que extract_dir foi criado e contém os arquivos do Passo 1
if not os.path.exists(extract_dir) or not os.listdir(extract_dir):
    print("ERRO: O diretório de extração está vazio ou não existe. O Passo 1 pode não ter sido bem-sucedido.")
else:
    for filename in os.listdir(extract_dir):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(extract_dir, filename)

            # Expressão regular mais flexível para CompanyCode e Year
            match = re.match(r'([A-Z0-9]+)_(\d{4}).*?\.xlsx', filename)

            company_code = None
            year = None

            if match:
                company_code = match.group(1)
                year = int(match.group(2))
            else:
                # Se ainda houver arquivos que não correspondem, isso será impresso.
                print(f"Aviso: Não foi possível extrair CompanyCode e Year de {filename}. Verifique o formato do nome do arquivo.")

            try:
                df = pd.read_excel(file_path)
                df['OriginalFileName'] = filename
                df['CompanyCode'] = company_code
                df['Year'] = year
                all_data.append(df)
            except Exception as e:
                print(f"Erro ao ler o arquivo {filename}: {e}")

if all_data:
    consolidated_df = pd.concat(all_data, ignore_index=True)
    print(f"DataFrame consolidado criado com {consolidated_df.shape[0]} linhas e {consolidated_df.shape[1]} colunas.")
    print("\nVerificação de valores nulos nas novas colunas após a concatenação:")
    print(consolidated_df[['OriginalFileName', 'CompanyCode', 'Year']].isnull().sum())
    print("\nPrimeiras 5 linhas do DataFrame consolidado, incluindo as novas colunas:")
    print(consolidated_df[['OriginalFileName', 'CompanyCode', 'Year']].head())
else:
    print("Nenhum dado foi carregado no DataFrame consolidado. Verifique os arquivos Excel.")

DataFrame consolidado criado com 915039 linhas e 8 colunas.

Verificação de valores nulos nas novas colunas após a concatenação:
OriginalFileName    0
CompanyCode         0
Year                0
dtype: int64

Primeiras 5 linhas do DataFrame consolidado, incluindo as novas colunas:
  OriginalFileName CompanyCode  Year
0  EMNIS_2009.xlsx       EMNIS  2009
1  EMNIS_2009.xlsx       EMNIS  2009
2  EMNIS_2009.xlsx       EMNIS  2009
3  EMNIS_2009.xlsx       EMNIS  2009
4  EMNIS_2009.xlsx       EMNIS  2009


In [4]:
# Certifique-se de que 'consolidated_df' existe do Passo 2
if 'consolidated_df' in locals() and not consolidated_df.empty:
    print("\nIniciando pré-processamento...")
    print("\nValores ausentes por coluna (antes do tratamento):")
    print(consolidated_df.isnull().sum()[consolidated_df.isnull().sum() > 0])

    # Exemplo de conversão de tipos de dados para colunas financeiras
    # Adapte esta lista com os nomes reais das suas colunas financeiras
    # Supondo que as colunas Desc, Yıllık, 9 Aylık, 6 Aylık, 3 Aylık são as colunas de valores
    financial_cols_to_check = ['Yıllık', '9 Aylık', '6 Aylık', '3 Aylık']

    for col in financial_cols_to_check:
        if col in consolidated_df.columns:
            # Converte para numérico, coercing errors para NaN
            consolidated_df[col] = pd.to_numeric(consolidated_df[col], errors='coerce')
            # Opcional: Preencher NaN em colunas numéricas com 0 ou a média, ou outro valor
            # consolidated_df[col].fillna(0, inplace=True)
        else:
            print(f"Aviso: Coluna '{col}' não encontrada para conversão de tipo.")

    # Exemplo de Feature Engineering: Criando uma razão financeira
    # (Adapte os nomes das colunas aos nomes reais do seu dataset)
    try:
        # Se você tem 'CurrentAssets' e 'CurrentLiabilities' ou similares:
        # if 'CurrentAssets' in consolidated_df.columns and 'CurrentLiabilities' in consolidated_df.columns:
        #     consolidated_df['CurrentRatio'] = consolidated_df['CurrentAssets'] / consolidated_df['CurrentLiabilities']
        #     print("\nColuna 'CurrentRatio' criada.")
        # else:
        #     print("\nColunas para CurrentRatio não encontradas.")
        print("Engenharia de Features (Exemplos) concluída ou pulada.")
    except Exception as e:
        print(f"\nErro durante a engenharia de features: {e}. Verifique os nomes das colunas e seus tipos de dados.")

    print("\nInformações do DataFrame após pré-processamento (tipos de dados):")
    print(consolidated_df.info())
else:
    print("DataFrame consolidado não disponível para pré-processamento. Verifique o Passo 2.")


Iniciando pré-processamento...

Valores ausentes por coluna (antes do tratamento):
Yıllık      14919
9 Aylık    125664
6 Aylık     98518
3 Aylık    148512
dtype: int64
Engenharia de Features (Exemplos) concluída ou pulada.

Informações do DataFrame após pré-processamento (tipos de dados):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915039 entries, 0 to 915038
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Desc              915039 non-null  object 
 1   Yıllık            900120 non-null  float64
 2   9 Aylık           789375 non-null  float64
 3   6 Aylık           816521 non-null  float64
 4   3 Aylık           766527 non-null  float64
 5   OriginalFileName  915039 non-null  object 
 6   CompanyCode       915039 non-null  object 
 7   Year              915039 non-null  int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 55.8+ MB
None


In [5]:
consolidated_df

,Desc,Yıllık,9 Aylık,6 Aylık,3 Aylık,OriginalFileName,CompanyCode,Year
0,Dönen Varlıklar,9457559.0,12071214.0,15581108.0,8897663.0,EMNIS_2009.xlsx,EMNIS,2009
1,Nakit ve Nakit Benzerleri,1610426.0,23911.0,14434.0,48297.0,EMNIS_2009.xlsx,EMNIS,2009
2,Finansal Yatırımlar,0.0,0.0,0.0,0.0,EMNIS_2009.xlsx,EMNIS,2009
3,Ticari Alacaklar,5026287.0,8501402.0,11668645.0,5436925.0,EMNIS_2009.xlsx,EMNIS,2009
4,Finans Sektörü Faaliyetlerinden Alacaklar,0.0,0.0,0.0,0.0,EMNIS_2009.xlsx,EMNIS,2009
...,...,...,...,...,...,...,...,...
915034,Diğer Nakit Girişi/Çıkışı,0.0,0.0,0.0,0.0,DGNMO_2022.xlsx,DGNMO,2022
915035,Nakit ve Benzerlerindeki Değişim,380981383.0,78669533.0,89823875.0,-23447193.0,DGNMO_2022.xlsx,DGNMO,2022
915036,Diğer Nakit ve Nakit Benzerlerindeki Artış,0.0,0.0,0.0,0.0,DGNMO_2022.xlsx,DGNMO,2022
915037,Dönem Başı Nakit Değerler,140545109.0,49168829.0,49168829.0,49168829.0,DGNMO_2022.xlsx,DGNMO,2022


In [6]:
consolidated_df['CompanyCode'].unique()

array(['EMNIS', 'VKFYO', 'KOTON', 'KRTEK', 'PNSUT', 'SNPAM', 'GLYHO',
       'PAGYO', 'ULAS', 'OSTIM', 'FENER', 'VESBE', 'CEMTS', 'ONRYT',
       'INTEM', 'IHLAS', 'YAYLA', 'DGGYO', 'GSDHO', 'ECILC', 'YUNSA',
       'OTKAR', 'ATEKS', 'BRKSN', 'ETYAT', 'AKSGY', 'CELHA', 'KLMSN',
       'OZKGY', 'SOKM', 'IHGZT', 'YESIL', 'TRILC', 'POLHO', 'IHAAS',
       'CWENE', 'QUAGR', 'EMKEL', 'TUPRS', 'BRSAN', 'ENSRI', 'DITAS',
       'BERA', 'VERTU', 'KLKIM', 'ZEDUR', 'ESEN', 'OSMEN', 'TEZOL',
       'MPARK', 'DZGYO', 'KRGYO', 'IZFAS', 'AVOD', 'PGSUS', 'MERIT',
       'CRFSA', 'INDES', 'AYGAZ', 'BLCYT', 'DGATE', 'EGEPO', 'AYCES',
       'ADEL', 'ISDMR', 'DOBUR', 'PRKAB', 'EKSUN', 'LYDHO', 'SKTAS',
       'ESCAR', 'MSGYO', 'DOHOL', 'EGPRO', 'VKING', 'CCOLA', 'PRZMA',
       'BANVT', 'KLSYN', 'CEMAS', 'SEGMN', 'MEPET', 'ISGYO', 'YGYO',
       'YAPRK', 'GENTS', 'ISMEN', 'BURCE', 'ARSAN', 'FONET', 'SURGY',
       'AEFES', 'SELGD', 'GOLTS', 'DOAS', 'KERVN', 'KMPUR', 'LINK',
       'LILAK', 'RYGYO', 'CME

In [7]:
import sqlite3

db_file = 'balancos.sqlite'

if 'consolidated_df' in locals() and not consolidated_df.empty:
    try:
        conn = sqlite3.connect(db_file)
        consolidated_df.to_sql('balancos_financeiros', conn, if_exists='replace', index=False)
        conn.close()
        print(f"\nDados armazenados com sucesso em '{db_file}' na tabela 'balancos_financeiros'.")

        # Opcional: Verificar se as novas colunas foram salvas corretamente
        conn_check = sqlite3.connect(db_file)
        # Seleciona as novas colunas e algumas outras para verificar
        df_check = pd.read_sql_query("SELECT OriginalFileName, CompanyCode, Year FROM balancos_financeiros LIMIT 5;", conn_check)
        print("\nVerificação das novas colunas CompanyCode, Year e OriginalFileName:")
        print(df_check.head())
        conn_check.close()

    except Exception as e:
        print(f"\nErro ao armazenar os dados no SQLite: {e}")
else:
    print("\nNão há dados consolidados para armazenar.")


Dados armazenados com sucesso em 'balancos.sqlite' na tabela 'balancos_financeiros'.

Verificação das novas colunas CompanyCode, Year e OriginalFileName:
  OriginalFileName CompanyCode  Year
0  EMNIS_2009.xlsx       EMNIS  2009
1  EMNIS_2009.xlsx       EMNIS  2009
2  EMNIS_2009.xlsx       EMNIS  2009
3  EMNIS_2009.xlsx       EMNIS  2009
4  EMNIS_2009.xlsx       EMNIS  2009
